# **OpenStreeMap Data Wrangling with MongoDB  **#

MapZen: https://mapzen.com/data/metro-extracts/metro/new-orleans_louisiana/

I used the New Orleans, LA, USA area for this project.  The zipped file is included in the source under the map directory.

## Load libraries needed for this project.

In [1]:
# load libraries
from collections import defaultdict
import codecs
import json
import re
import os
import pprint
import datetime as dt
import csv
import pandas as pd
import xml.etree.cElementTree as cET
import signal
import subprocess
from pymongo import MongoClient

#Load OSM file and path, in this case we are looking at New Orleans, LA, USA
filename = "new-orleans_louisiana.osm"
#Sample File to test with
#filename = "sample.osm"
path = "./map"
osmFile = os.path.join(path, filename)

## Review Data

We'll take a cursory look at the data by looking at the elements, users, and potential problems with the data.

In [2]:
#count the number of unique elements in our file to get a picture of the data structure
def count_tags(osm):
        tags = {}
        for event, elem in cET.iterparse(osm):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
#execute count_tags on our osm file
tags = count_tags(osmFile)
#print the results
pprint.pprint(tags)

{'bounds': 1,
 'member': 68224,
 'nd': 7124566,
 'node': 6413023,
 'osm': 1,
 'relation': 5963,
 'tag': 1664974,
 'way': 378498}


In [3]:
#get the users that have contributed to this OSM file
def get_users(osm):
    users = set()
    for _, element in cET.iterparse(osm):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = get_users(osmFile)
len(users)

899

In [4]:
lower_case = re.compile(r'^([a-z]|_)*$')
colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
badchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#search tags for three types of elements, ones that value values that are all lowercase and are valid, ones that are valid but have colons, and one that looks for bad or problem characters
def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower_case.search(k):
                keys['lower_case'] += 1
            elif colon.search(k):
                keys['colon'] += 1
            elif badchars.search(k):
                keys['badchars'] += 1
            else:
                keys['other'] += 1
    return keys


def search_keys(osm):
    keys = {"lower_case": 0, "colon": 0, "badchars": 0, "other": 0}
    for _, element in cET.iterparse(osm):
        keys = key_type(element, keys)

    return keys

keys = search_keys(osmFile)
pprint.pprint(keys)

{'badchars': 3, 'colon': 690509, 'lower_case': 659110, 'other': 315352}


## Data Problems And Cleanup

We need to clean up the address abbreviations so naming is consistent.

In [5]:
#Clean up street names so they all use the same abbreviation, print out the ones we updated.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway",
                         "Place", "Road", "Square", "Street", "Trail"]
mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square"}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

def update_street_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

def audit_streets(osmfile):
    #take inventory of street names
    streets = audit(osmfile)
    #loops through and show the ones that were updated
    street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
    for street_type, ways in streets.iteritems():
        for name in ways:
            updated_name = update_street_name(name, mapping, street_type_re)
            if updated_name != name :
                print name, "=>", updated_name
                
#audit street names
audit_streets(osmFile)

Banks St => Banks Street
Octavia St => Octavia Street
S Broad St => S Broad Street
Ponce De Leon St => Ponce De Leon Street
Chartres St => Chartres Street
Magazine Street;Magazine St => Magazine Street;Magazine Street
East St => East Street
E Rutland St => E Rutland Street
North St => North Street
Marais St => Marais Street
Laurel St => Laurel Street
N Rendon St => N Rendon Street
Magazine St => Magazine Street
Lasalle St => Lasalle Street
Tchoupitoulas St => Tchoupitoulas Street
South St => South Street
Toulouse St => Toulouse Street
Dauphine St => Dauphine Street
Decatur St => Decatur Street
543 5th St => 543 5th Street
Royal St => Royal Street
Bourbon St => Bourbon Street
Canal St => Canal Street
South St Patrick St => South St Patrick Street
Whitney St => Whitney Street
S Dupre St => S Dupre Street
Prytania St => Prytania Street
Johnny Dufrene St => Johnny Dufrene Street
Bayou Rd => Bayou Road
West I-10 Service Rd => West I-10 Service Road
Cobia Dr => Cobia Drive
Acadia Dr => Acadi

### Clean up invalid zipcodes

In [6]:
def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]

    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)

    elif twoDigits != 95:
        invalid_zipcodes[twoDigits].add(zipcode)

def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

def update_zips(zipcode):
    testNum = re.findall('[a-zA-Z]*', zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip()
    if testNum == "LA":
        convertedZipcode = (re.findall(r'\d+', zipcode))
        if convertedZipcode:
            if convertedZipcode.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]
    return zipcode


def audit_zips(osmFile):
    zipcodes = audit(osmFile)

    for street_type, ways in zipcodes.iteritems():
        for zipcode in ways:
            updated_zipcode = update_zips(zipcode)
            if zipcode != updated_zipcode :
                print zipcode, "=>", updated_zipcode
                
#audit zip codes
audit_zips(osmFile)

LA 70116 => 70116
LA 70117 => 70117


## Convert and import data into MongoDB

In [7]:
#Convert the file to json and save
CREATED = [ "version", "changeset", "timestamp", "user", "uid" ]

def shape_element(element) :
    node = {}
    if (element.tag == "node") or (element.tag == "way") :
        node["created"]={}
        node["pos"]=[]
        node["id"]=element.attrib["id"]
        node["type"]=element.tag

        try:
            node["pos"].append(float(element.attrib["lat"]))
            node["pos"].append(float(element.attrib["lon"]))
        except:
            pass
        for a in CREATED:

            if a in element.attrib:
                node["created"][a]=element.attrib[a]

        temp_address = {}
        
        if element.findall("./tag")!=[]:
            node["feature"]={}
            for tag in element.iter("tag"):
                if tag.attrib["k"] == "addr:street":
                    street = update_street_name(tag.attrib["v"], mapping, street_type_re)
                    if street:
                        temp_address["street"] = street
                elif tag.attrib["k"] == "addr:postcode":
                    zipcode = update_zips(tag.attrib["v"])
                    if zipcode:
                        temp_address["postcode"] = zipcode
                else:
                    node["feature"][tag.attrib["k"]]=tag.attrib["v"]

            if temp_address:
                node["address"] = temp_address
    return  node

def convert_file(file_in):

    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in cET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                fo.write(json.dumps(el) + "\n")
    return data

convert_file(osmFile)

[{'created': {'changeset': '46303647',
   'timestamp': '2017-02-22T11:24:34Z',
   'uid': '4007051',
   'user': 'upendrakarukonda',
   'version': '15'},
  'feature': {'ele': '2',
   'is_in:continent': 'North America',
   'is_in:country': 'United States',
   'is_in:country_code': 'US',
   'loc_name:vi': u'Ng\u1ecdc L\xe2n',
   'name': 'New Orleans',
   'name:ca': 'Nova Orleans',
   'name:es': 'Nueva Orleans',
   'name:fr': u'La Nouvelle-Orl\xe9ans',
   'name:kn': u'\u0cb9\u0cca\u0cb8 \u0c92\u0cb0\u0ccd\u0cb2\u0cc0\u0ca8\u0ccd\u0cb8\u0ccd',
   'name:ru': u'\u041d\u043e\u0432\u044b\u0439 \u041e\u0440\u043b\u0435\u0430\u043d',
   'name:zh': u'\u65b0\u5965\u5c14\u826f',
   'place': 'city',
   'population': '354850',
   'wikidata': 'Q34404',
   'wikipedia': 'en:New Orleans'},
  'id': '28110976',
  'pos': [29.9499323, -90.0701156],
  'type': 'node'},
 {'created': {'changeset': '2204087',
   'timestamp': '2009-08-19T19:12:09Z',
   'uid': '147510',
   'user': 'woodpeck_fixbot',
   'version': '3'

In [8]:
#connect to mongo db
def connectToMongo() :
    pro = subprocess.Popen('mongod', preexec_fn = os.setsid)
    db_name = 'udacity_p3'
    mongo = MongoClient('localhost:27017')
    return mongo[db_name]

db = connectToMongo()



In [9]:
#Import our file into mongodb
db_name = 'udacity_p3'
collection = 'data'
json_file = osmFile + '.json'

mongoimport_cmd = 'mongoimport -h 127.0.0.1:27017 ' + \
                  '--db ' + db_name + \
                  ' --collection ' + collection + \
                  ' --file ' + json_file

# Before importing, drop collection if it is already running 
if collection in db.collection_names():
    print 'Dropping collection: ' + collection
    db[collection].drop()
    
# Execute the command
print 'Executing: ' + mongoimport_cmd
subprocess.call(mongoimport_cmd.split())

Dropping collection: data
Executing: mongoimport -h 127.0.0.1:27017 --db udacity_p3 --collection data --file ./map/new-orleans_louisiana.osm.json


0

## Load DB object for querying

Load the db object for querying and explore the data

In [10]:
new_orleans_la_data = db[collection]

### Get size of the original OSM file and the new JSON file

In [11]:
print 'The original OSM file is {} MB'.format(os.path.getsize(osmFile)/1.0e6) # convert from bytes to megabytes
print 'The JSON file is {} MB'.format(os.path.getsize(osmFile + ".json")/1.0e6) # convert from bytes to megabytes


The original OSM file is 1278.919201 MB
The JSON file is 1388.497993 MB


### Number of documents

In [12]:
new_orleans_la_data.find().count()

6791521

### Number of unique users

In [13]:
len(new_orleans_la_data.distinct('created.user'))

885

### Number of nodes and ways

In [14]:
print "Number of nodes:",new_orleans_la_data.find({'type':'node'}).count()
print "Number of ways:",new_orleans_la_data.find({'type':'way'}).count()

Number of nodes: 6413023
Number of ways: 378498


### Top 10 Contributors to this data

In [15]:
result = new_orleans_la_data.aggregate( [
                                        { "$group" : {"_id" : "$created.user", 
                                        "count" : { "$sum" : 1} } },
                                        { "$sort" : {"count" : -1} }, 
                                        { "$limit" : 10 } ] )

print(list(result))

[{u'count': 3441210, u'_id': u'Matt Toups'}, {u'count': 1226743, u'_id': u'ELadner'}, {u'count': 769110, u'_id': u'wvdp'}, {u'count': 348774, u'_id': u'coleman_nolaimport'}, {u'count': 250871, u'_id': u'ELadnerImp'}, {u'count': 212231, u'_id': u'woodpeck_fixbot'}, {u'count': 53381, u'_id': u'Matt Toups_nolaimport'}, {u'count': 38526, u'_id': u'Minh Nguyen_nolaimport'}, {u'count': 36321, u'_id': u'ceseifert_nolaimport'}, {u'count': 25930, u'_id': u'Maarten Deen'}]


### Top 10 Amenities

In [16]:

amenities=new_orleans_la_data.aggregate([{"$match":{"feature.amenity":{"$exists":1}}},
                       {"$group":{"_id":"$feature.amenity","count":{"$sum":1}}},
                       {"$sort":{"count":-1}},
                       {"$limit":10}
                      ])

for result in amenities:
    print(result)

{u'count': 1132, u'_id': u'place_of_worship'}
{u'count': 846, u'_id': u'school'}
{u'count': 558, u'_id': u'parking'}
{u'count': 408, u'_id': u'restaurant'}
{u'count': 182, u'_id': u'grave_yard'}
{u'count': 180, u'_id': u'fast_food'}
{u'count': 167, u'_id': u'kindergarten'}
{u'count': 154, u'_id': u'fire_station'}
{u'count': 143, u'_id': u'fuel'}
{u'count': 106, u'_id': u'cafe'}


### Top Religion 

In [17]:
religion=new_orleans_la_data.aggregate([{"$match":{"feature.amenity":"place_of_worship"}},
                       {"$group":{"_id":"$feature.religion","count":{"$sum":1}}},
                       {"$sort":{"count":-1}},
                       {"$limit":1}
                      ])
for result in religion:
    print result

{u'count': 1084, u'_id': u'christian'}


### Most common types of cuisine

In [18]:
restaurants=new_orleans_la_data.aggregate([{"$match":{"feature.amenity":"restaurant"}},
                       {"$group":{"_id":"$feature.cuisine","count":{"$sum":1}}},
                       {"$sort":{"count":-1}},
                       {"$limit":10}
                      ])
for result in restaurants:
    print(result)


{u'count': 236, u'_id': None}
{u'count': 22, u'_id': u'regional'}
{u'count': 16, u'_id': u'mexican'}
{u'count': 15, u'_id': u'pizza'}
{u'count': 13, u'_id': u'american'}
{u'count': 8, u'_id': u'vietnamese'}
{u'count': 8, u'_id': u'italian'}
{u'count': 8, u'_id': u'chinese'}
{u'count': 8, u'_id': u'seafood'}
{u'count': 6, u'_id': u'japanese'}


### Most common types of fast food

In [19]:
fast_food=new_orleans_la_data.aggregate([{"$match":{"feature.amenity":"fast_food"}},
                       {"$group":{"_id":"$feature.cuisine","count":{"$sum":1}}},
                       {"$sort":{"count":-1}},
                       {"$limit":10}
                      ])
for result in fast_food:
    print(result)


{u'count': 81, u'_id': None}
{u'count': 43, u'_id': u'burger'}
{u'count': 16, u'_id': u'sandwich'}
{u'count': 12, u'_id': u'chicken'}
{u'count': 8, u'_id': u'american'}
{u'count': 6, u'_id': u'pizza'}
{u'count': 4, u'_id': u'mexican'}
{u'count': 2, u'_id': u'ice_cream'}
{u'count': 1, u'_id': u'seafood'}
{u'count': 1, u'_id': u'sushi'}


### Top 10 Gas Stations

In [20]:
def get_gas_stations(data):
    gas_stations=data.aggregate([{"$match":{"feature.amenity":"fuel"}},
                       {"$group":{"_id":"$feature.name","count":{"$sum":1}}},
                       {"$sort":{"count":-1}},
                       {"$limit":10}
                      ])
    for result in gas_stations:
        print(result)
        
get_gas_stations(new_orleans_la_data)

{u'count': 81, u'_id': None}
{u'count': 17, u'_id': u'Shell'}
{u'count': 8, u'_id': u'Chevron'}
{u'count': 4, u'_id': u'Exxon'}
{u'count': 3, u'_id': u'Racetrack'}
{u'count': 2, u'_id': u'Circle K'}
{u'count': 1, u'_id': u'Kangaroo'}
{u'count': 1, u'_id': u'Circle K Shell'}
{u'count': 1, u'_id': u'Cheap Smokes'}
{u'count': 1, u'_id': u'Interstate Fuel'}


## Improvements

One way to improve the data is to get more people involved in collecting and verifying the data.  My recommendation is to create a mobile application that can interact and engage with people from their smart devices.

I would use react native to build this app.  React Native allows you build apps in javascript that can compile into both Android and iOS applications.   This ability is beneficial so that one person doesn't have to learn two distinct programming languages, or does have to hire extra people to build 2 separate apps.  It also will cut down on time spent on maintenance and upgrades. React Native also has a geolocation feature that will allow us to know the users position which will be vital in tracking this information.

When the user is stationary for a while, the app should prompt the user to enter information about their current location.  If it is a location that has been previously entered, it could prompt to verify the information.  The should be asked for the name of the location, as well as details such as type of location and address.   We can prompt for further information based on the type of location being entered.

To provide better and more information, we could also tie into the Google Maps API.  We can use the latitude and longitude from our geolocation package to pull data from google.  It would be possible that in many instances a user could just be prompted to confirm information instead of having to enter it.  This would make for a much better experience and the ease of use should mean more interactions with users.

To increase user engagement, we will need to incentivize our users through gamification.  I found a great resource (listed below) that listed the benefits of a bage based reward system in elearning.  I also think about the Achievement system in a game such as World of Warcraft.  These systems create a competitive environment for even trivial tasks.  It can reward the users with recognition via leaderboards and also help give them goals to work for.  Some ideas would be badges for entering 10, 25, 50, and 100 (to start) locations.  


https://facebook.github.io/react-native/
https://facebook.github.io/react-native/docs/geolocation.html
https://enterprise.google.com/maps/products/mapsapi.html
https://elearningindustry.com/notable-benefits-using-badge-based-elearning-reward-system
http://www.wowhead.com/achievement-guides

## Anticipated Issues

There are downsides to my proposal above. 

1.  Building an app will cost time and mooney regardless of the technology used.  Apple and Google both charge fees to considered an official developer and publish apps to their respective stores.  Someone will need to learn react native and build the application (time) or hire someone to do it (money).

2.  Depending on users to enter data can easily lead to unreliable and conflicting data.  We would need a process in place to validate the data to make sure we have the most accurate data available published.  This would be another good use of the Google Maps API.  

3.  Google Maps would be a great tool to integrate, however, it could become pricey.  I think our app would be free to use, which would start us at the standard plan.  However, you get a limited amount of API calls for free.  As the popularity of the app rises, the more this will cost.  Also, integrating and maintaining that integration with a 3rd party api will consume a lot of time.  If someone is hired to do this, it will also increase the cost monetarily.

4.  Gamification will also cost time and money.  The badges will require design and graphic work.  We would want these to look good and professional.  Integrating them into the app and adding the badge system will increase the complexity of the code.  That will be more time in implementation, maintenance, and testing.  

https://enterprise.google.com/maps/pricing/#compare-editions

